In [38]:
import pandas as pd
import string
from glove import Glove

In [64]:
# columns to use
cols=['description', 'points']

# import data
reviews_1 = pd.read_csv('../../data/wine-reviews/winemag-data_first150k.csv', index_col=False, usecols=cols)
reviews_2 = pd.read_csv('../../data/wine-reviews/winemag-data-130k-v2.csv', index_col=False, usecols=cols)

print("Number of entries in dataset 1: %s" %reviews_1.shape[0])
print("Number of entries in dataset 2: %s" %reviews_2.shape[0])

duplicates = set(reviews_1.description).intersection(set(reviews_2.description))

print("\nNumber of duplicate entries across datasets: %s" % len(duplicates))

# concatenate and drop duplicates
data = pd.concat([reviews_1,reviews_2])
data.drop_duplicates(inplace=True)

print("\nNumber of unique reviews: %s" % data.shape[0])

data.head()

Number of entries in dataset 1: 150930
Number of entries in dataset 2: 129971

Number of duplicate entries across datasets: 48346

Number of unique reviews: 169461


,description,points
0,This tremendous 100% varietal wine hails from ...,96
1,"Ripe aromas of fig, blackberry and cassis are ...",96
2,Mac Watson honors the memory of a wine once ma...,96
3,"This spent 20 months in 30% new French oak, an...",96
4,"This is the top wine from La Bégude, named aft...",95


In [135]:
punc_remove = string.punctuation
punc_remove = punc_remove.replace('%', '')
table = str.maketrans(dict.fromkeys(punc_remove))

# lowercase
data['description_test'] = data.description_test.str.lower()
# remove punctuation
data['description_test'] = data.description_test.str.translate(table)
# replace percentage sign
data['description_test'] = data.description_test.str.replace('%', ' percent')
# split words
data['description_test'] = data.description_test.str.split()

In [137]:
data.head()

,description,points,description_test
0,This tremendous 100% varietal wine hails from ...,96,"[this, tremendous, 100, percent, varietal, win..."
1,"Ripe aromas of fig, blackberry and cassis are ...",96,"[ripe, aromas, of, fig, blackberry, and, cassi..."
2,Mac Watson honors the memory of a wine once ma...,96,"[mac, watson, honors, the, memory, of, a, wine..."
3,"This spent 20 months in 30% new French oak, an...",96,"[this, spent, 20, months, in, 30, percent, new..."
4,"This is the top wine from La Bégude, named aft...",95,"[this, is, the, top, wine, from, la, bégude, n..."


In [180]:
import tensorflow as tf
from collections import Counter
import math

In [184]:
def create_vocabulary(documents):
    """Unique words and counts"""
    vocabulary = Counter()

    for row in documents:
        vocabulary.update(row)
        
    return vocabulary

vocabulary = create_vocabulary(data.description_test)
vocabulary_size = len(vocabulary)

print("Number of unique words: %s" % vocabulary_size)

Number of unique words: 58541


In [186]:
def top_vocabulary(vocabulary, n_words=10000):
    """Limit vocabulary to highest occurring words."""
    count = [['UNK', -1]]
    count.extend(vocabulary.most_common(n_words - 1))
    
    return count

test = vocabulary_common(vocabulary)

[['UNK', -1],
 ('and', 453109),
 ('the', 289722),
 ('a', 236677),
 ('of', 217846),
 ('with', 161893),
 ('this', 139508),
 ('is', 125881),
 ('wine', 101023),
 ('flavors', 84121),
 ('in', 82508),
 ('to', 72651),
 ('it', 72445),
 ('its', 64978),
 ('fruit', 60416),
 ('on', 57045),
 ('that', 49200),
 ('aromas', 47439),
 ('but', 46261),
 ('finish', 44385),
 ('palate', 43611),
 ('acidity', 42039),
 ('from', 39298),
 ('tannins', 38987),
 ('cherry', 34999),
 ('drink', 34453),
 ('ripe', 33514),
 ('are', 32590),
 ('has', 31817),
 ('black', 30912),
 ('for', 30432),
 ('by', 28802),
 ('spice', 24422),
 ('an', 24285),
 ('as', 24140),
 ('notes', 23671),
 ('red', 23176),
 ('rich', 22934),
 ('percent', 22734),
 ('dry', 22418),
 ('oak', 22117),
 ('fresh', 20870),
 ('nose', 20714),
 ('now', 20164),
 ('berry', 19519),
 ('sweet', 18740),
 ('soft', 18419),
 ('plum', 17289),
 ('blackberry', 16546),
 ('blend', 16279),
 ('fruits', 16130),
 ('crisp', 16123),
 ('shows', 15852),
 ('very', 15314),
 ('apple', 15312)

In [177]:
# word embeddings

# latent features
embedding_size = 50
vocabulary_size = len(vocabulary)
# init values
embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))

# initialize weights and biases for word2vect model
# each unique word gets a weight per latent feature and a single bias
nce_weights = tf.Variable(
  tf.truncated_normal([vocabulary_size, embedding_size],
                      stddev=1.0 / math.sqrt(embedding_size)))
nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

In [178]:
# Placeholders for inputs (reviews are read in in batches during training)
batch_size = 25

train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])

NameError: name 'vocabulary_size' is not defined